![](ubc_header.png)

# Climate Friendly Food System (CFFS) Labelling Pilot
***

## Objective

Implement the Climate-Friendly and Just Food System (CFFS &Just) definition at the UBC Campus by producing the weighted metric that informs the choice of icon for each menu item.

## Data Source

**Recipes Data**: extracted from UBC culinary services management platform Optimum Control
    
    Items.xml: Raw ingredient Items
    Ingredients.xml: Ingredients/Preps that goes into prep/products recipes
    Preps.xml: Preps that go into final product recipes
    Products.xml: Final Recipes/Menu Items
    Conversions.xml: Unit Conversion Information

**cool_food_ghg.csv**: extracted from the [Cool Food Calculator](https://coolfood.org/pledge/). For this project, we use emission factors for the active total supply chain emissions in the North America region. The food category is at the detailed level for higher results accuracy

## Assumptions

* The same GHG emission factor will be assigned to different forms (puree, sliced, chopped etc) of the same raw ingredient

* We assume the GHG factors for different varieties of one ingredient are the same (i.e Red/Yellow Onion)

* The loss/add of ingredient weight during the cooking process will be ignored

* The amount of unusable part and wastes during ingredients processing process are ignored 

* Ignore GHG emissions from the cooking process

* Assume the GHG emission factor for water is zero, and ignore the water use in the cooking process


## Import Libraries

In [30]:
#!pip3 install pdpipe

In [31]:
import numpy as np 
import pandas as pd
import pdpipe as pdp
import matplotlib.pyplot as plt
import glob
import os

import csv
from itertools import islice

In [32]:
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse

In [33]:
import openpyxl

## Load Data

#### Data Path

In [34]:
filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "oc", "*.oc"))

In [35]:
filepath_list

['/Users/silvia/cffs-label/data analysis/data/oc/IPR_Export_03152021_1121.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/IPR_Export_03152021_1346.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/IPR_Export_03122021_0915.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/IPR_Export_03152021_1224.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/IPR_Export_03152021_1250.oc']

#### Load Items List

In [36]:
ItemId = []
Description = []
Cost = []
CaseQty = []
CaseUOM = []
PakQty = []
PakUOM = []
InventoryGroup = []

for filepath in filepath_list:
    xtree = et.parse(filepath + '/items.xml')
    xroot = xtree.getroot()
    for item in xtree.iterfind('Item'):
        ItemId.append(item.attrib['id'])
        Description.append(item.findtext('Description'))
        Cost.append(item.findtext('Cost'))
        CaseQty.append(item.findtext('CaseQty'))
        CaseUOM.append(item.findtext('CaseUOM'))
        PakQty.append(item.findtext('PakQty'))
        PakUOM.append(item.findtext('PakUOM'))
        InventoryGroup.append(item.findtext('InventoryGroup'))

        
Items = pd.DataFrame({'ItemId': ItemId, 'Description': Description, 'Cost': Cost, 'CaseQty': CaseQty, 
                      'CaseUOM': CaseUOM, 'PakQty': PakQty, 'PakUOM': PakUOM, 'InventoryGroup': InventoryGroup}
                    ).drop_duplicates()

Items.head()

,ItemId,Description,Cost,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-4472,AVOCADO MX,1.2450,20.000,CT,1.000,CT,PRODUCE
1,I-4973,AVOCADO PULP CHUNKY,0.0114,12.000,bag,454.000,g,PRODUCE
2,I-27410,BACON 3MM NATURALLY SMKD,9.0000,5.000,Kg,1.000,Kg,MEAT
3,I-1286,BAGEL WHOLE WHEAT (6 PACK),0.5000,6.000,CT,1.000,CT,BREAD
4,I-3130,BASE BEEF CONC LIQ G/FREE,0.0151,4.000,pak,946.000,ml,FOOD - GROCERY


In [37]:
Items.shape

(390, 8)

In [38]:
path = os.path.join(os.getcwd(), "data", "processed", "Items_List.csv")
Items.to_csv(path, index = False, header = True)

#### Load Ingredient List

In [39]:
IngredientId = []
Conversion = []
InvFactor = []
Qty = []
Recipe = []
Uom = []

for filepath in filepath_list:
    xtree = et.parse(filepath + '/Ingredients.xml')
    xroot = xtree.getroot()
    for x in xtree.iterfind('Ingredient'):
        IngredientId.append(x.attrib['ingredient'])
        Conversion.append(x.attrib['conversion'])
        InvFactor.append(x.attrib['invFactor'])
        Qty.append(x.attrib['qty'])
        Recipe.append(x.attrib['recipe'])
        Uom.append(x.attrib['uom'])
    
Ingredients = pd.DataFrame({'IngredientId': IngredientId, 'Conversion': Conversion, 
                      'InvFactor': InvFactor, 'Qty': Qty, 'Recipe': Recipe,'Uom': Uom}).drop_duplicates()

Ingredients.head()

,IngredientId,Conversion,InvFactor,Qty,Recipe,Uom
0,I-3388,1.00000000,0.3058,1.000,P-10496,L
1,I-4660,2.20462000,0.6942,2.270,P-10496,Kg
2,I-4679,1.00000000,0.0063,1.000,P-18318,BUNCH
3,I-4793,2.20462000,1.2048,10.000,P-18746,Kg
4,I-3643,0.00100000,0.1837,225.000,P-18907,g


In [40]:
Ingredients.shape

(2198, 6)

In [41]:
path = os.path.join(os.getcwd(), "data", "processed", "Ingredients_List.csv")
Ingredients.to_csv(path, index = False, header = True)

#### Load Preps List

In [42]:
PrepId = []
Description = []
Cost = []
PakQty = []
PakUOM = []
InventoryGroup = []

for filepath in filepath_list:
    xtree = et.parse(filepath + '/Preps.xml')
    xroot = xtree.getroot()
    for x in xtree.iterfind('Prep'):
        PrepId.append(x.attrib['id'])
        Description.append(x.findtext('Description'))
        Cost.append(x.findtext('Cost'))
        PakQty.append(x.findtext('PakQty'))
        PakUOM.append(x.findtext('PakUOM'))
        InventoryGroup.append(x.findtext('InventoryGroup'))
    
Preps = pd.DataFrame({'PrepId': PrepId, 'Description': Description, 'Cost': Cost,
                  'PakQty': PakQty, 'PakUOM':PakUOM, 'InventoryGroup': InventoryGroup}).drop_duplicates()

Preps.head()

,PrepId,Description,Cost,PakQty,PakUOM,InventoryGroup
0,P-56398,BATCH| Guacamole AR,11.7873,2.500,Kg,PREP
1,P-26333,BATCH|Citrus Herb Aioli,5.7669,3.000,L,PREP
2,P-35713,BLACKENED|Chicken,1.5686,185.000,ea,PREP
3,P-51579,BREADED|Chicken|Cooked,7.5905,1.600,Kg,
4,P-50831,BREADED|Chicken|Karaage Thigh,8.9681,4.500,Kg,


In [43]:
Preps.shape

(427, 6)

In [44]:
path = os.path.join(os.getcwd(), "data", "processed", "Preps_List.csv")
Preps.to_csv(path, index = False, header = True)

#### Load Product List

In [45]:
ProdId = []
Description = []
Cost = []

for filepath in filepath_list:
    xtree = et.parse(filepath + '/Products.xml')
    xroot = xtree.getroot()
    for x in xtree.iterfind('Prod'):
        ProdId.append(x.attrib['id'])
        Description.append(x.findtext('Description'))
        Cost.append(x.findtext('Cost'))
        
Products = pd.DataFrame({'ProdId': ProdId, 'Description': Description, 'Cost': Cost}).drop_duplicates()

Products.head()

,ProdId,Description,Cost
0,R-36666,FF|Bowl|Five Alarm,2.2206
1,R-36568,FF|Pasta|Spinach Avocado,1.7066
2,R-50362,FLX|Bowl|Karaage Chicken,3.0860
3,R-50360,FLX|Bowl|Salmon,4.3323
4,R-50359,FLX|Bowl|Tuna Albacore,3.6203


In [46]:
Products.shape

(103, 3)

In [47]:
path = os.path.join(os.getcwd(), "data", "processed", "Products_List.csv")
Products.to_csv(path, index = False, header = True)

#### Load Conversion List

In [48]:
ConversionId = []
Multiplier = []
ConvertFromQty = []
ConvertFromUom = []
ConvertToQty = []
ConvertToUom = []

for filepath in filepath_list:
    xtree = et.parse(filepath + '/Conversions.xml')
    xroot = xtree.getroot()
    for x in xtree.iterfind('Conversion'):
        ConversionId.append(x.attrib['id'])
        Multiplier.append(x.attrib['multiplier'])
        ConvertFromQty.append(x.find('ConvertFrom').attrib['qty'])
        ConvertFromUom.append(x.find('ConvertFrom').attrib['uom'])
        ConvertToQty.append(x.find('ConvertTo').attrib['qty'])
        ConvertToUom.append(x.find('ConvertTo').attrib['uom'])
    
    
Conversions = pd.DataFrame({'ConversionId': ConversionId, 'Multiplier': Multiplier, 'ConvertFromQty': ConvertFromQty,
                           'ConvertFromUom': ConvertFromUom, 'ConvertToQty': ConvertToQty, 'ConvertToUom': ConvertToUom}
                          ).drop_duplicates()

Conversions.head()

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,,1.00000000,1.0000,XXX,1.0000,L
1,,0.87719298,1.0000,1.14L,1.1400,L
2,,0.66666667,1.0000,1.5L,1.5000,L
3,,0.57142857,1.0000,1.75 L,1.7500,L
4,,0.50000000,1.0000,2L,2.0000,L


In [49]:
Conversions.shape

(240, 6)

In [50]:
path = os.path.join(os.getcwd(), "data", "processed", "Products_List.csv")
Products.to_csv(path, index = False, header = True)

#### Load Emission Factors

In [51]:
cfc_filename = os.path.join(os.getcwd(), "data", "raw", "cool_food_ghg.csv")
cfc = pd.read_csv(cfc_filename)
cfc.head()

,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,1,beef & buffalo meat,41.3463
1,2,lamb/mutton & goat meat,41.6211
2,3,pork (pig meat),9.8315
3,4,"poultry (chicken, turkey)",4.3996
4,5,butter,11.4316


In [52]:
cfc.shape

(54, 3)

## Data Analysis

#### Mapping Ingredients to GHG Factors

In [58]:
# Load Items List with assigned Cetegory ID
items = pd.read_csv(os.path.join(os.getcwd(), "data", "processed", "Items_List_Assigned.csv"))

In [59]:
df1 = pd.DataFrame(items)
df2 = pd.DataFrame(cfc)

In [60]:
df1.head()

,ItemId,CategoryID,Description,Cost,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-37002,1.0,BEEF INSIDE ROUND SHAVED,0.0158,9.00,Kg,1000.0,g,MEAT
1,I-37005,1.0,BEEF MEATBALLS,0.0093,4.54,Kg,1000.0,g,MEAT
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,8.4900,1.00,Kg,1.0,Kg,MEAT
3,I-10869,1.0,BEEF STIRFRY COV FR,14.5000,5.00,Kg,1.0,Kg,MEAT
4,I-52090,1.0,BURGER BEEF & MUSHROOM HALAL,1.4742,1.00,cs,48.0,CT,MEAT


In [61]:
df2.head()

,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,1,beef & buffalo meat,41.3463
1,2,lamb/mutton & goat meat,41.6211
2,3,pork (pig meat),9.8315
3,4,"poultry (chicken, turkey)",4.3996
4,5,butter,11.4316


In [63]:
mapping = pd.merge(df1, df2.loc[:,['Category ID','Food Category','Active Total Supply Chain Emissions (kg CO2 / kg food)']], 
                  how = 'left',
                  left_on = 'CategoryID', 
                  right_on = 'Category ID')

In [64]:
mapping.head()

,ItemId,CategoryID,Description,Cost,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,I-37002,1.0,BEEF INSIDE ROUND SHAVED,0.0158,9.00,Kg,1000.0,g,MEAT,1.0,beef & buffalo meat,41.3463
1,I-37005,1.0,BEEF MEATBALLS,0.0093,4.54,Kg,1000.0,g,MEAT,1.0,beef & buffalo meat,41.3463
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,8.4900,1.00,Kg,1.0,Kg,MEAT,1.0,beef & buffalo meat,41.3463
3,I-10869,1.0,BEEF STIRFRY COV FR,14.5000,5.00,Kg,1.0,Kg,MEAT,1.0,beef & buffalo meat,41.3463
4,I-52090,1.0,BURGER BEEF & MUSHROOM HALAL,1.4742,1.00,cs,48.0,CT,MEAT,1.0,beef & buffalo meat,41.3463


In [65]:
mapping.shape

(390, 12)

#### Calculate GHG Emissions